In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

import pickle
import pandas as pd
from stapep.molecular_dynamics import PrepareProt, Simulation
from stapep.utils import PhysicochemicalPredictor, ProtParamsSeq

In [2]:
seq = 'RLIQ-R8-LEDVEK-S5-RKV'
output = 'data'

# Prepare the protein for molecular dynamics simulation
pp = PrepareProt(seq, output, 'alphafold')
pp._gen_prmtop_and_inpcrd_file()

sim = Simulation(output)
sim.setup(
        type='implicit', # 'explicit' or 'implicit'
        solvent='water', # 'water' or 'chloroform'
        temperature=300, # Kelvin
        friction=1, # ps^-1
        timestep=2, # fs
        interval=1000,
        nsteps=5000000 # 10 ns
    )
sim.minimize()
sim.run()

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-I: Adding /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/prep to search path.
-I: Adding /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/lib to search path.
-I: Adding /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/parm to search path.
-I: Adding /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /home/dahuilangda/Simulation/miniconda3/envs/stap/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading ti

In [3]:
# Initialize the ProtParamsSeq class
pps = ProtParamsSeq(seq)

# Initialize the PhysicochemicalPredictor class, which automatically loads trajectory using pytraj.
pcp = PhysicochemicalPredictor(sequence=seq, 
                                topology_file=os.path.join(output, 'pep_vac.prmtop'),
                                trajectory_file=os.path.join(output, 'traj.dcd'),
                                start_frame=500)

# Save the mean structure of the trajectory
pcp._save_mean_structure(os.path.join(output, 'mean_structure.pdb'))

data/traj.dcd data/pep_vac.prmtop


'data/mean_structure.pdb'

In [4]:
# We selected these features for machine learning
feature_cols = ['length', 'weight', 'helix_percent',
       'loop_percent', 'mean_bfactor', 'mean_gyrate', 'hydrophobic_index',
       'num_hbonds', 'charge', 'aromaticity', 'isoelectric_point',
       'fraction_arginine', 'fraction_lysine', 'psa']

# calculate the features
length = pps.seq_length
weight = pps.weight
helix_percent = pcp.calc_helix_percent()
loop_percent = pcp.calc_loop_percent()
mean_bfactor = pcp.calc_mean_bfactor()
mean_gyrate = pcp.calc_mean_gyrate()
hydrophobic_index = pcp.calc_hydrophobic_index(os.path.join(output, 'mean_structure.pdb'))
num_hbonds = pcp.calc_n_hbonds()
charge = pps.calc_charge(pH=7.0)
aromaticity = pps.aromaticity
isoelectric_point = pps.isoelectric_point
fraction_arginine = pps.fraction_arginine
fraction_lysine = pps.fraction_lysine
psa = pcp.calc_psa(os.path.join(output, 'mean_structure.pdb'))

# put the features into a DataFrame
df = pd.DataFrame([[length, weight, helix_percent, loop_percent, 
                    mean_bfactor, mean_gyrate, hydrophobic_index, num_hbonds, 
                    charge, aromaticity, isoelectric_point, fraction_arginine, 
                    fraction_lysine, psa]], columns=feature_cols)

In [5]:
# load the machine learning model
model = pickle.load(open('../models/rf_cls_model.pkl', 'rb'))

# predict the permeability
permeability_prob = model.predict_proba(df[feature_cols])[0][1]
permeability_ = 'permeable' if permeability_prob > 0.5 else 'impermeable'
print(f'The permeability of "{seq}" is "{permeability_} ({round(permeability_prob,3)})".')

The permeability of "RLIQ-R8-LEDVEK-S5-RKV" is "impermeable (0.27)".
